# Week 3 - Data Warehouse

**Setup**

* Create an external table using the FHV tripdata from year 2019.
* Create a table in BigQuery using the FHV tripdata from year 2019 (do not partition nor cluster this table).

Answer:
1. Create an external table using the data
```sql
CREATE OR REPLACE EXTERNAL TABLE `stellarismusv2.trips.fhv_tripdata_ext`
OPTIONS (
  format = 'CSV',
  uris = ['gs://raw-a43b47dc95/fhv/*.csv.gz']
);
```

2. Create a table in BigQuery using the FHV tripdata from year 2019

```sql
CREATE OR REPLACE TABLE `stellarismusv2.trips.fhv_tripdata` AS
SELECT * FROM `stellarismusv2.trips.fhv_tripdata_ext`;
```

**Question 1. What is the count for the FHV tripdata records for year 2019?**
- 65,623,481
- 43,244,696
- 22,978,333
- 13,942,414
Answer:
```sql
SELECT     COUNT(1)
FROM       `stellarismusv2.trips.fhv_tripdata`;
>> 43244696
```

**Question 2. Write a query to count the number of distinct values in affiliated_base_number column for the entire dataset on both the tables. What is the estimated amount of data that will be read when this query is executed on the External Table and the Table?**
- 25.2 MB for the External Table and 100.87MB for the BQ Table
- 225.82 MB for the External Table and 47.60MB for the BQ Table
- 0 MB for the External Table and 0MB for the BQ Table
- 0 MB for the External Table and 317.94MB for the BQ Table 
Answer:

- External Table:
```sql
SELECT     COUNT(DISTINCT(Affiliated_base_number))
FROM       `stellarismusv2.trips.fhv_tripdata_ext`;
>> Bytes processed : 0 B (results cached)
```
- Native Table: 
```sql
SELECT     COUNT(DISTINCT(Affiliated_base_number))
FROM       `stellarismusv2.trips.fhv_tripdata`;
>> Bytes processed : 317.94 MB
```

**Question 3. How many records have both NULL *PUlocationID* and *DOlocationID* in the entire dataset?**
* 717,748
* 1,215,687
* 5
* 20,332

Answer:

```sql
SELECT     COUNT(1)
FROM       `stellarismusv2.trips.fhv_tripdata`
WHERE      PUlocationID IS NULL
           AND DOlocationID IS NULL;
>> 717748
```

**Question 4. What is the best strategy to optimize the table if query always filter by pickup_datetime and order by affiliated_base_number?**
- Cluster on pickup_datetime Cluster on affiliated_base_number
- Partition by pickup_datetime Cluster on affiliated_base_number
- Partition by pickup_datetime Partition by affiliated_base_number
- Partition by affiliated_base_number Cluster on pickup_datetime

**Answer:** Partition by pickup_datetime Cluster on affiliated_base_number
Always partition on date and cluster for optimization. 


**Question 5. Implement the optimized solution you chose for question 4. Write a query to retrieve the distinct affiliated_base_number between pickup_datetime 2019/03/01 and 2019/03/31 (inclusive).**
Use the BQ table you created earlier in your from clause and note the estimated bytes. Now change the table in the from clause to the partitioned table you created for question 4 and note the estimated bytes processed. What are these values? Choose the answer which most closely matches.
- 12.82 MB for non-partitioned table and 647.87 MB for the partitioned table
- 647.87 MB for non-partitioned table and 23.06 MB for the partitioned table
- 582.63 MB for non-partitioned table and 0 MB for the partitioned table
- 646.25 MB for non-partitioned table and 646.25 MB for the partitioned table

Answer:

1. Create a partitioned and clustered table


```sql
CREATE OR REPLACE TABLE `stellarismusv2.trips.fhv_tripdata_optimized`
PARTITION BY DATE(pickup_datetime)
CLUSTER BY affiliated_base_number AS
SELECT * FROM `stellarismusv2.trips.fhv_tripdata_ext`;
```  
2. Scan Non-Optimized Table

```sql
SELECT COUNT(DISTINCT affiliated_base_number) FROM `stellarismusv2.trips.fhv_tripdata`
WHERE pickup_datetime BETWEEN "2019-03-01" AND "2019-03-31";
>> 647.87 MB
```  
3. Scan Optimized Table
```sql
SELECT COUNT(DISTINCT affiliated_base_number) FROM `stellarismusv2.trips.fhv_tripdata_optimized`
WHERE pickup_datetime BETWEEN "2019-03-01" AND "2019-03-31";
>> 23.05 MB
```           

**Question 6. Where is the data stored in the External Table you have created?**
- Big Query
- GCP Bucket
- Container Registry
- Big Table

**Answer:** GCP Bucket.



**Question 7. Is it true, that you should always cluster your data in BigQuery?**
- True
- False

**Answer:** False. It is an optimization technique with its trade offs. They also slow down loading which is not suitable for staging tables. It depends generally on query patterns, data size, presence of unique values and on need of accurate cost estimates.
